In [ ]:
MOD = 1000000

h2c0 = [0b01011010, 0b10011010, 0b11010010]                                     #높이가 2고 빈칸으로 시작하는 블럭의 모양(stack 모양임에 주의)
h2c1 = [0b00011011, 0b10011001, 0b01011001, 0b01001011, 0b10010011, 0b11001001] #높이가 2고 차있는 칸으로 시작하는 블럭의 모양
h3c0 = [0b00011110, 0b00111010, 0b00111100]
h3c1 = [0b00110011, 0b00010111, 0b00100111, 0b00111001, 0b00001111]

def go(N, M, DP, i, B) :
  if i >= N * M :
    if i == N * M and B == 0 : 
      return 1 #모든 칸을 지나가고 마지막 칸이 비어있는 경우
    return 0

  if DP[i][B] != -1 :
    return DP[i][B]

  res = 0 #현재의 i와 B에서 놓을 수 있는 블럭이 아무것도 없을 경우 반환값은 0이여야 한다.
  if B & 0b1 : #현재 칸이 채워져있는 경우, 현재 칸이 비어있을 때만 블럭을 놓을 수 있는 블럭을 놓는다.
    if i % M != 2 : #블럭의 모양이 망가지면 안된다.
      if i % M == 0 : #높이가 3칸인 블럭이 가능한 경우
        for mask in h3c0 :
          if B & mask : continue
          res += go(N, M, DP, i + 1, (B | mask) >> 1) #마스크를 씌운 다음에 한칸 민다.
          res %= MOD
      for mask in h2c0 :
        if B & mask : continue
        res += go(N, M, DP, i + 1, (B | mask) >> 1) 
        res %= MOD
    res += go(N, M, DP, i + 1, B >> 1) #현재 블럭이 채워져있는 경우 블럭을 놓지 않고 스킵할 수 있다.
    res %= MOD
  else : #그렇지 않은 경우 채워있을때만 가능한 블럭을 놓는다.
    if i % M != 2 :
      if i % M == 0 :
        for mask in h3c1 :
          if B & mask : continue
          res += go(N, M, DP, i + 1, (B | mask) >> 1)
          res %= MOD
      for mask in h2c1 :
        if B & mask : continue
        res += go(N, M, DP, i + 1, (B | mask) >> 1)
        res %= MOD
  DP[i][B] = res % MOD
  return DP[i][B]

def sol() :
  N = int(input())
  if N < 4 : return print(0)
  DP = [[-1] * (1 << 8) for _ in range(3 * N)]
  go(N, 3, DP, 0, 0)
  print(DP[0][0] % MOD)
  
sol()

### 접근
- 격자판 채우기 DP 유형일 것이라고 확신하고 접근했다.
- 판의 한쪽 길이가 3이므로 가능한 테트리스 블럭의 모양을 모두 고려할 수 있게끔 8칸의 칸을 보는 것으로 설정했다.

### 풀이(자력솔)
- 보기에 나온 테트리스 블럭의 모양에 대해, 한점을 고정하고 블럭들이 회전한 모든 경우를 전부 고려했더니 총 17가지의 경우가 나왔다.
  - 그중에서도 블럭을 구성하는데에 높이가 2칸이 필요한 경우와 3칸이 필요한 경우로 나뉜다는 것을 알 수 있었다.
    - 필요한 높이가 N칸 필요하다는 중요한데, 블럭의 모양을 망가뜨리면 안되기 때문이다. \
    이전 유형들에서 가로로 자르는 경우일 때 `i % M != (M-1)`을 조건을 확인한 이유에 대해서 생각하면 된다.
    - 또한 높이가 2인 경우에서 블럭의 시작점이 이 맨 위, 즉 `i % M`이 `0`일때와, `1`일때 두가지 경우를 모두 다르게 고려해야 될지도 모른다고 생각했지만,\
    직접 해본결과 놀랍게도 비트필드로 표현했을 때 두 경우의 비트필드는 똑같았다.
- DP의 재귀에 들어간 스킵 횟수와 상태 표현에 대해
  - 이전 문제에서 세로로 자른 경우에 i + 2로 이동하고, 상태도 두칸 쉬프트하는 부분이 신경쓰여서 좀 자세히 살펴봤다.
  - 그 결과 굳이 두칸씩 쉬프트를 할 필요는 없었다는 것을 알 수 있었다. +1에 한칸 쉬프트 했을 때, 어짜피 아무 연산도 하지 않고 또 스킵해야 하기 때문이다.
    - `res += go(N, M, DP, i + 1, B >> 1) #현재 블럭이 채워져있는 경우 블럭을 놓지 않고 스킵할 수 있다.` 이 부분의 코드 때문에 이것이 가능하다. 
  - 테트리스의 블럭 모양을 모두 고려하기엔 전처리가 너무 복잡해지기 때문에 이 사실을 이용해 i와 상태 모두 1칸씩 증가시키는 것으로 통일했다.
- 비트마스크 구현 관련
  - 예를들어, 어떤 8자리의 비트 필드에서 (뒤에서) 2, 3번째 bit가 1이고, 4번째 bit가 0인지 확인하는 방법은 다음과 같다.
  - `off_mask = 0b00000110`\
  `on_mask = 0b00001000`\
  `check = (B & off_mask) == 0 and (B & on_mask) != 0`
  